# ROCK: Reasoning About Commonsense Causality

Use this notebook for performing CCR using ROCK

In [1]:
from __future__ import print_function, absolute_import, division

%load_ext autoreload
%autoreload 2
%matplotlib widget

import sys, os, json, time, datetime, logging, warnings, multiprocessing, itertools
import tqdm, json, sqlite3, ast
from pathlib import Path
import pandas as pd
import numpy as np
import torch
import nltk, spacy
import transformers, allennlp
from transformers import (AutoTokenizer, AutoModelForMaskedLM,
                          RobertaModel,RobertaForMaskedLM, 
                          RobertaTokenizer, GPT2LMHeadModel, GPT2Tokenizer)
import allennlp_models
import allennlp_models.pretrained

In [2]:
import src
import src.pipeline
import src.utils as utils


In [3]:
print(torch.cuda.is_available())
TORCH_DEV = torch.device(f'cuda:0') if torch.cuda.is_available() \
                                    else torch.device("cpu")

logging.getLogger('allennlp.common.params').disabled = True 
logging.getLogger('allennlp.nn.initializers').disabled = True 
logging.getLogger('allennlp.modules.token_embedders.embedding').setLevel(logging.INFO) 
logging.getLogger('urllib3.connectionpool').disabled = True 
logging.getLogger().setLevel(logging.CRITICAL)
warnings.filterwarnings('ignore')
logging.disable(sys.maxsize)

True


In [4]:
def console_log(msg, end='\n'):
    os.write(1, ('[LOG/{}]'.format(multiprocessing.current_process().name)+msg+end).encode('utf-8'))


def col_print(*args, cw=12, sep='|'):
    print(f" {sep} ".join(('{'+f":<{cw}"+'}').format(s) for s in args))
    
def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    
def set_ts_seed():
    set_seed(int(str(datetime.datetime.now().timestamp()).replace('.', '')) % (2 ** 31))

[Process path `from pathlib import Path`](https://www.freecodecamp.org/news/how-to-use-pathlib-module-in-python/#:~:text=Concrete%20Paths%20in%20Python,being%20in%20that%20operating%20system.)

In [5]:
DATA_PATH = Path("./exp_data")
MODEL_PATH = Path("./models")

In [6]:
# set_seed(hsh('random_string') % (2 ** 31))
set_ts_seed()

## Load Dataset

In [7]:
copa_dev = pd.read_json(DATA_PATH / "copa_dev.json", lines=True, orient='records').set_index('idx')
copa_test = pd.read_csv(DATA_PATH / "copa_test.json")
glt_d1 = pd.read_csv(DATA_PATH / "glucose_d1_probs.csv")

In [8]:
copa_dev.head()

,premise,choice1,choice2,question,label
idx,,,,,
0,The man turned on the faucet.,The toilet filled with water.,Water flowed from the spout.,effect,1
1,The girl found a bug in her cereal.,She poured milk in the bowl.,She lost her appetite.,effect,1
2,The woman retired.,She received her pension.,She paid off her mortgage.,effect,0
3,I wanted to conserve energy.,I swept the floor in the unoccupied room.,I shut off the light in the unoccupied room.,effect,1
4,The hamburger meat browned.,The cook froze it.,The cook grilled it.,cause,1


In [9]:
copa_test.head()

,premise,choice1,choice2,question,label
0,The item was packaged in bubble wrap.,It was fragile.,It was small.,cause,0
1,I emptied my pockets.,I retrieved a ticket stub.,I found a weapon.,effect,0
2,Termites invaded the house.,The termites disappeared from the house.,The termites ate through the wood in the house.,effect,1
3,The travelers reached the border.,The patrol agent checked their passports.,The patrol agent accused them of smuggling.,effect,0
4,The office was closed.,It was a holiday.,It was summer.,cause,0


In [10]:
glt_d1.head()

,selected_sentence,candidates,answer,answer0,answer1,answer2,text,covariates,interventions,index,outcome,answer_idx,covariates_cleaned,p_xd,p_dy,p_xy
0,They won regionals and were on the way to the ...,['The football team had worked hard all season...,The football team had worked hard all season. ...,The football team had worked hard all season.,They won regionals and were on their way to th...,"During practice, the quarterback broke his arm.",The football team had worked hard all season.,"[""The football team had worked hard all season...",['The football team was new and worked hard al...,0,They won regionals and were on their way to th...,1,"[""He'd had to work even harder to overcome his...","[[(0.0734076276421547, 0.11186903121415526, 0....","[(0.46431438624858856, 0.4771946966648102), (0...","[(0.3315032720565796, 0.39027543365955353), (0..."
1,They won regionals and were on the way to the ...,['The football team had worked hard all season...,The football team had worked hard all season. ...,The football team had worked hard all season.,They won regionals and were on their way to th...,"During practice, the quarterback broke his arm.",The football team had worked hard all season.,"[""The football team had worked hard all season...",['The football team was new and worked hard al...,0,"During practice, the quarterback broke his arm.",2,"[""He'd had to work even harder to overcome his...","[[(0.0734076276421547, 0.11186903121415526, 0....","[(0.5384128838777542, 0.4292968362569809), (0....","[(0.2735405806452036, 0.187699181959033), (0.2..."
2,The team had to use the second string quarterb...,['The football team had worked hard all season...,The quarterback breaks his arm >Causes/Enables...,The quarterback breaks his arm,The team has to use The second string quarterback,"Luckily, the team still won the play-offs.",The quarterback breaks his arm,"['The quarterback breaks his arm Before that, ...","['A player breaks his arm', 'A receiver breaks...",1,The team has to use The second string quarterback,1,"['He had a cyst on his side, he broke his wri...","[[(0.4629475921392441, 0.47641928493976593, 0....","[(0.4776032269001007, 0.5128596127033234), (0....","[(0.18337566778063774, 0.20733415335416794), (..."
3,The team had to use the second string quarterb...,['The football team had worked hard all season...,The quarterback breaks his arm >Causes/Enables...,The quarterback breaks his arm,The team has to use The second string quarterback,"Luckily, the team still won the play-offs.",The quarterback breaks his arm,"['The quarterback breaks his arm Before that, ...","['A player breaks his arm', 'A receiver breaks...",1,"Luckily, the team still won the play-offs.",2,"['He had a cyst on his side, he broke his wri...","[[(0.4629475921392441, 0.47641928493976593, 0....","[(0.42544446885585785, 0.5650285333395004), (0...","[(0.3369382694363594, 0.3855547308921814), (0...."
4,I practiced all the time to be really good.,['When I was a kid I wanted to be really good ...,i want to be good at playing violin >Causes/En...,i want to be good at playing violin,i practices a lot,After a few months of practicing I did get rea...,i want to be good at playing violin,['i want to be good at playing violin Before t...,"[""the door's a little higher than i want to be...",2,i practices a lot,1,"['I was good at playing the flute,.', 'I want ...","[[(0.2706519067287445, 0.2139797806739807, 0.0...","[(0.45580950379371643, 0.5197682678699493), (0...","[(0.24025794863700867, 0.25716912746429443), (..."


# ROCK Pipeline

[spacy en_core_web_md](https://spacy.io/models/en)

In [11]:
# !python -m spacy download en_core_web_md 
## First download the model!

In [12]:
spacy_model = spacy.load('en_core_web_md')
allensrl = src.pipeline.AllenSRLWrapper(allennlp_models.pretrained.load_predictor("structured-prediction-srl-bert", cuda_device=0))

## Temporal Predictor

As a bare minimum, a customized temporal predictor needs to overwrite the `predict` method.
Below is the implmentation we used based on mask language modeling.

```python
class TempPredictor:
    def __init__(self, model, tokenizer, device, spacy_model="en_core_web_sm"):
        self._model = model
        self._model.to(device)
        self._model.eval()
        self._tokenizer = tokenizer
        self._mtoken = self._tokenizer.mask_token
        self.unmasker = transformers.pipeline("fill-mask", model=self._model, tokenizer=self._tokenizer, device=0)
        try:
            self._spacy = spacy.load(spacy_model)
        except Exception as e:
            self._spacy = spacy.load("en_core_web_sm")
            print(f"Failed to load spacy model {spacy_model}, use default 'en_core_web_sm'\n{e}")


    def predict(self, e1, e2, top_k=5):
        """
        returns
        """
        txt = self._remove_punct(e1) + " " + self._mtoken + " " + self._sent_lowercase(e2)
        return self.unmasker(txt, top_k=top_k)


    def get_temp(self, e1, e2, top_k=5, crop=1):
        inst1 = self.predict(e1, e2, top_k)
        inst2 = self.predict(e2, e1, top_k)

        # e1 before e2
        b1 = self._extract_token_prob(inst1, "before", crop=crop)
        b2 = self._extract_token_prob(inst2, "after", crop=crop)

        # e1 after e2
        a1 = self._extract_token_prob(inst1, "after", crop=crop)
        a2 = self._extract_token_prob(inst2, "before", crop=crop)

        return (b1+b2)/2, (a1+a2)/2

    def __call__(self, *args, **kwargs):
        return self.get_temp(*args, **kwargs)
    
    # other methods omitted
```

**NB** Fine-tuned RoBERTa model checkpoint can be downloaded using [this anonymous Dropbox link](https://www.dropbox.com/s/9egrzn1ny3oq2qa/roberta_ft.tar.gz?dl=0) (1.29GB).

In [13]:
tp_roberta_ft = src.pipeline.TempPredictor(
    model=RobertaForMaskedLM.from_pretrained(MODEL_PATH/"roberta_ft"),
    tokenizer=RobertaTokenizer.from_pretrained("roberta-base"),
    device=TORCH_DEV
)

tp_roberta_base = src.pipeline.TempPredictor(
    model=RobertaForMaskedLM.from_pretrained("roberta-base"),
    tokenizer=RobertaTokenizer.from_pretrained("roberta-base"),
    device=TORCH_DEV
)

##### Sanity Check

In [17]:
utils.test_copa_run(copa_dev.iloc[0], tp_roberta_base, tp_roberta_ft, top_k=5)

Premise: The man turned on the faucet.
C1: The toilet filled with water.
C2: Water flowed from the spout.
Question: effect	Correct choice: Choice 2

============== PREMISE <---> CHOICE 1
The man turned on the faucet. <---> The toilet filled with water.
Base model:	before: 0.081	after: 0.017
FT model:	before: 0.456	after: 0.498

============== PREMISE <---> CHOICE 2 (expect before > after)
The man turned on the faucet. <---> Water flowed from the spout.
Base model:	before: 0.107	after: 0.007
FT model:	before: 0.525	after: 0.472


In [23]:
count = 0
for idx in range(len(copa_dev)):
    premise, choice1, choice2, q, lb = copa_dev.iloc[idx][['premise', 'choice1', 'choice2', 'question', 'label']]
    predictor = tp_roberta_base
    res = [predictor.get_temp(premise, choice1, top_k=5), predictor.get_temp(premise, choice2, top_k=5)]
    r = res[lb]
    if (q == "effect" and r[0] > r[1]) or (q == "cause" and r[0] < r[1]):
        count += 1

count / len(copa_dev)

0.61

In [24]:
count = 0
for idx in range(len(copa_dev)):
    premise, choice1, choice2, q, lb = copa_dev.iloc[idx][['premise', 'choice1', 'choice2', 'question', 'label']]
    predictor = tp_roberta_ft
    res = [predictor.get_temp(premise, choice1, top_k=5), predictor.get_temp(premise, choice2, top_k=5)]
    r = res[lb]
    if (q == "effect" and r[0] > r[1]) or (q == "cause" and r[0] < r[1]):
        count += 1

count / len(copa_dev)

0.63

## Event Sampler

The event sampler subclasses `EventGenerator`.
As a bare minimum, a custom implmentation should provide the `__call__` method.

```python
class EventGenerator:
    def __init__(self, model, tokenizer, spacy_model, device):
        self.model = model.to(device)
        self.tokenizer = tokenizer
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.device = device

    def __call__(self, prompt, max_length=30, **kwargs):
        # pass
```

Below is our wrapper for GPT-J that is used in our paper:


```python

class GPTJGenerator:
    def __init__(self, model, tokenizer, device=None):

        self.model = model
        
        if device is not None:
            self.model = self.model.to(device)

        self.tokenizer = tokenizer
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.device = device

    def __call__(self, prompt, **kwargs):
        output_id = self.model.generate(self.tokenizer(prompt, return_tensors="pt", padding=True).input_ids, **kwargs)
        return self.tokenizer.batch_decode(output_id)

```

In [25]:
from transformers import AutoTokenizer, AutoModelForCausalLM
gptj_tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")
gptj_model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-j-6B")

Downloading:   0%|          | 0.00/22.5G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
gpt_generator = src.pipeline.GPTJGenerator(model=gptj_model, tokenizer=gptj_tokenizer)

In [ ]:
gen_kwargs = dict(max_length=30,
                  do_sample=True,
                  temperature=0.9,
                  num_return_sequences=10,
                 )

In [ ]:
gen_sents = gpt_generator("The man turned on the faucet.", **gen_kwargs)

In [ ]:
print("\n".join(gen_sents))

The man turned on the faucet. His face was a mask of concentration, and his hands were steady as he washed the car.


The man turned on the faucet. It was a small plastic one, nothing fancy. His hands shook, but he managed to turn the water
The man turned on the faucet. He drank from it until the bottle was empty.

After a moment he pulled the cap off the
The man turned on the faucet. Water began splashing into the sink. He was washing the dishes, his expression blank, his eyes dead
The man turned on the faucet.

Bruno had to take a step back. The water made a sound. The man looked
The man turned on the faucet. The water gushed out. A loud splash, and water went everywhere, flowing down the man's body
The man turned on the faucet. He watched it fill with cold water. He heard it gurgle into the sink. He watched it
The man turned on the faucet. He scrubbed down the sink and the tub, washed the toilet bowl, then flushed. In the meantime
The man turned on the faucet. A strong current of wa

## Interventions

As a bare minimum, the intervention generator should
implement `__call__` method that takes a prompt and additional
kwargs as arguments and return a list of interventions.

```python
class InterventionGenerator:
    def __init__(self, **kwargs):
        pass
    
    def __call__(self, prompt, **kwargs:)
        pass

```


We use PolyJuice in our implementation based on their implementation [0].


[0] https://github.com/tongshuangwu/polyjuice

In [26]:
cf_gen = src.pipeline.PJGenerator(srl_processor=allensrl)


Downloading:   0%|          | 0.00/828 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/487M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
cf_texts = cf_gen("The man turned off the faucet.",
      ctrl_codes=[
          "resemantic", 
          "negation", 
          "lexical",
          "quantifier"
          "insert",
          "restructure",
          "shuffle",
          "delete"
                 ]
                 )

In [ ]:
print("\n".join(cf_texts['resemantic']))

The woman turned off the faucet.
The man turned off the faucet.
The man poured water from the watering can into the pitcher until the watering can was empty. off the faucet.
The man replaced the bell brand off the faucet.
The man lit the cigarette off the faucet.
The man turned off the water main.
The man turned off the dishwasher.
The man turned off the fan and replaced it with a fan outside of his house.


## Processing Datasets for Evaluation

### Construct DataFrames for processing

In [ ]:
def gen_copa_proc_df(copa):
    return pd.DataFrame(list(itertools.chain.from_iterable(
        [[s[0], 'premise', s[1]['premise']]] if s[1]['question'] == 'effect'
        else [[s[0], 'choice1', s[1]['choice1']], [s[0], 'choice2', s[1]['choice2']]]
        for s in copa.iterrows()
    )), columns=['index', 'name', 'text'])

In [ ]:
copa_proc = gen_copa_proc_df(copa_dev)

In [ ]:
copa_proc.head()

,index,name,text
0,0,premise,The man turned on the faucet.
1,1,premise,The girl found a bug in her cereal.
2,2,premise,The woman retired.
3,3,premise,I wanted to conserve energy.
4,4,choice1,The cook froze it.


We can apply the components row by row, but it is more efficient to let
each component batch process the data

#### Sample Covariates

In [ ]:
def sample_cov(df, model, tokenizer):
    output_ids = []
    for s in df.iterrows():
        prompt = f"{s[1]['text']} Before that, "

        gen_tokens = model.generate(tokenizer(prompt,
                          return_tensors="pt", padding=True).input_ids, 
                    do_sample=True,
                    temperature=0.9,
                    max_length=30,
                    num_return_sequences=100,
            )
        output_ids.append(gen_tokens)
    return [tokenizer.batch_decode(tks) for tks in output_ids]



In [ ]:
copa_proc['covariates'] = sample_cov(copa_proc, gptj_model, gptj_tokenizer)

#### Generating Interventions

In [ ]:
def get_interventions(self, s, cf_gen, **kwargs):
    interventions = self.cf_gen(s, gen_kwargs=kwargs)
    intvers = list(itertools.chain(*[ints for _, ints in interventions.items()]))
    self.last_gen['interventions'] = intvers
    return intvers

In [ ]:
copa_proc['interventions'] = copa_proc.apply(lambda s : get_interventions(s, cf_gen, 
                                ctrl_codes=[
                                      "resemantic", 
                                      "negation", 
                                      "lexical",
                                      "quantifier"
                                      "insert",
                                      "restructure",
                                      "shuffle",
                                      "delete"
                                ], axis=1)

### Obtain Temporal Probabilities

In [ ]:
# use `utils.glt_get_probs`
# if working on glucose-d1
copa_proc = copa_proc.apply(lambda s : utils.copa_get_probs(s, model=tp_roberta_ft, 
                                                            top_k=5, spacy_model=spacy_model), 
                            axis=1)



#### Add a few columns

In [ ]:
def postproc_copa(df):
    df['label_idx'] = df['name'].apply(
        lambda s: -1 if s == 'premise' else int(s[-1])-1
    )

    df['outcome'] = df.apply(lambda s:
        None if s['label_idx'] == -1 else copa_test.iloc[s['index']]['premise'], axis=1)


    tmp_df = df[df['label_idx']==-1].copy()
    tmp_df['label_idx'] = 1
    df.loc[df['label_idx']==-1, 'label_idx']=0
    df = pd.concat([df, tmp_df])
    return df

In [ ]:
copa_proc = postproc_copa(df)

#### Save Data

In [ ]:
copa_proc.head()

,index,name,text,covariates,interventions,outcome,label_idx,p_xd,p_dy,p_xy
0,0,premise,The man turned on the faucet.,"[""The man turned on the faucet. Before that, h...","['The man chose to turned on the faucet.', 'At...",The toilet filled with water.,0,"[[(0.210379958152771, 0.30398909747600555, 0.0...","[(0.45566828548908234, 0.4977114349603653), (0...","[(0.04539947956800461, 0.03326283395290375), (..."
1,0,premise,The man turned on the faucet.,"[""The man turned on the faucet. Before that, h...","['The man chose to turned on the faucet.', 'At...",Water flowed from the spout.,1,"[[(0.210379958152771, 0.30398909747600555, 0.0...","[(0.5247508734464645, 0.4717450588941574), (0....","[(0.3355148509144783, 0.25840914994478226), (0..."
2,1,premise,The girl found a bug in her cereal.,"[""The girl found a bug in her cereal. Before t...",['While digging for well water Monica stayed o...,She poured milk in the bowl.,0,"[[(0.5899830460548401, 0.39629100263118744, 0....","[(0.47232694923877716, 0.5270598828792572), (0...","[(0.5464454889297485, 0.40834908187389374), (0..."
3,1,premise,The girl found a bug in her cereal.,"[""The girl found a bug in her cereal. Before t...",['While digging for well water Monica stayed o...,She lost her appetite.,1,"[[(0.5899830460548401, 0.39629100263118744, 0....","[(0.4848470687866211, 0.5141351819038391), (0....","[(0.6019861996173859, 0.38734038174152374), (0..."
4,2,premise,The woman retired.,"[""The woman retired. Before that, she'd writte...","['Suddenly the woman retired.', 'The author ra...",She received her pension.,0,"[[(0.3462740257382393, 0.30608220398426056, 0....","[(0.4986240118741989, 0.49911610782146454), (0...","[(0.5325719714164734, 0.45344893634319305), (0..."


In [ ]:
copa_proc.to_csv(DATA_PATH/"copa_dev_probs.csv")

## Next Steps

Please see `result_presentation.ipynb` notebook for evaulation.